In [1]:
import os
import sys
import argparse
import itertools
import subprocess
import pandas as pd
import multiprocessing
from libs import functions
from datetime import datetime
from multiprocessing import Pool
from subprocess import run, PIPE
import re
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
def interaction_calc(seq):
    proc = run(['RNAup', '-b','-o','--interaction_first'], stdout=PIPE,stderr=subprocess.DEVNULL,
               input=str.encode(seq)) #input is a stdin object so encode input str
    return str(proc.stdout).replace("\\n"," ").replace("b'","")

In [3]:
mrna = functions.fasta_to_dataframe("test.fa")
ncrna = functions.fasta_to_dataframe("test_ncrna.fa")
mrna['mrna_seq'] = '>' + mrna[1].map(str) + '\n' + mrna[0].map(str).str[:30] + '\n'
ncrna['ncrna_seq'] = '>' + ncrna[1].map(str) + '\n' + ncrna[0].map(str) + '\n'
mrna_seq = [rows['mrna_seq'] for index,rows in mrna.iterrows()]   
ncrna_seq = [rows['ncrna_seq'] for index,rows in ncrna.iterrows()]
index = pd.MultiIndex.from_product([mrna_seq , ncrna_seq], names = ['mrna', 'ncrna'])
sequence_df = pd.DataFrame(index = index).reset_index()
df = sequence_df.pivot(index='mrna',columns='ncrna',values='ncrna')
df['first'] = df.reset_index().values.sum(axis=1)

In [4]:
total_pairs = df.shape[0]
my_pool = Pool(4)
result = my_pool.imap_unordered(interaction_calc, df['first'], chunksize = int(total_pairs/4))
my_pool.close()
my_pool.join()

In [5]:
result = list(result)
seq_id = pd.Series(result).str.extractall(r'(>\w+-\w+)')
ncrna_id = (seq_id.loc[pd.IndexSlice[:, 1:2], :]).reset_index().set_index('level_0')
mrna_id = (seq_id.loc[pd.IndexSlice[:, 0], :]).reset_index().set_index('level_0')
mfe = pd.Series(result).str.extractall(r'(\(-[0-9]+\.[0-9]+)')[0].str.replace('(', '', regex=True).to_frame().reset_index().set_index('level_0')

In [6]:
d = pd.concat([mrna_id, ncrna_id, mfe], axis=1)
d = d.iloc[:,[1,3,5]]
d.columns = ['mRNA', 'ncRNA', 'MFE']

In [7]:
d

,mRNA,ncRNA,MFE
level_0,,,
0,>NYSGXRC-10001e_10,>ECD_RS01040_tRNA-Ile_GAT,-5.73
0,>NYSGXRC-10001e_10,>ECD_RS19040_tRNA-Trp_CCA,-5.92
1,>NYSGXRC-10001g_1,>ECD_RS01040_tRNA-Ile_GAT,-6.13
1,>NYSGXRC-10001g_1,>ECD_RS19040_tRNA-Trp_CCA,-4.40
2,>NYSGXRC-10001d_1,>ECD_RS01040_tRNA-Ile_GAT,-12.24
2,>NYSGXRC-10001d_1,>ECD_RS19040_tRNA-Trp_CCA,-5.22
3,>NYSGXRC-10001c_12,>ECD_RS01040_tRNA-Ile_GAT,-6.83
3,>NYSGXRC-10001c_12,>ECD_RS19040_tRNA-Trp_CCA,-5.10
4,>NYSGXRC-10001i_1,>ECD_RS01040_tRNA-Ile_GAT,-5.67
